In [ ]:
import mne
import numpy as np
from mne.preprocessing import ICA, corrmap
from mne_icalabel import label_components
from eutils.preproc.preprocess import data_loader, set_info, montage_dict, fs_dict

In [ ]:
dataset_name = "KUL"
sub_id = "1"
l_freq = 1
h_freq = 32
label_type = 'direction'

data, label = data_loader(dataset_name, sub_id, label_type)

In [ ]:
# ica_dict = {'KUL': [0, 2], 'DTU': ['eog1', 'eog2', 'eog3', 'eog4', 'eog5', 'eog6'], 'SCUT': [0, 1]}
ica_dict = {'KUL': [0, 2], 'DTU': [0, 1, 7], 'SCUT': [0, 1]}  # 没有50Hz去工频电的ica
# ica_dict = {'KUL': [], 'DTU': [0, 5], 'SCUT': []}  # 去掉工频电后的ica

# 准备电极信息
info = set_info(dataset_name)

# 加载模板数据（S1-Trail1）
data_tmp, label_tmp = data_loader(dataset_name, '1', label_type)
data_tmp = np.transpose(data_tmp, (0, 2, 1))
data_tmp = data_tmp[0]

# 计算ica通道
raw_tmp = mne.io.RawArray(data_tmp, info)
raw_tmp = raw_tmp.filter(l_freq=1, h_freq=None)
raw_tmp.set_montage(montage_dict[dataset_name])
ica_tmp = ICA(n_components=20, max_iter='auto', random_state=97)
ica_tmp.fit(raw_tmp)

# 去眼电
is_verbose = True
data = np.transpose(data, (0, 2, 1))
# for k_tra in range(data.shape[0]):
k_tra = 0
print(f'data ica, trail: {k_tra}')

# 将原始数据转化为raw格式文件
raw = mne.io.RawArray(data[k_tra], info, verbose=is_verbose)

# 计算ica数据
raw = raw.filter(l_freq=1, h_freq=None, verbose=is_verbose)
ica = ICA(n_components=20, max_iter='auto', random_state=97, verbose=is_verbose)  # 97为随机种子
ica.fit(raw)

In [ ]:
ic_labels = label_components(raw, ica, method="iclabel")